
# 연습 문제1 — 인터넷뱅킹 관련 월별 시계열 분석(한국은행(ECOS))

**안내 **

 
`api_key` 변수에 본인의 한국은행 ECOS API 키를 준비하세요.

---


### 요구 사항

통계표 코드(stat_code): 631Y001 (internet banking 관련 지표)

주기(frequency): 연간(A) — 연도(YYYY) 단위로 제공 (월별 데이터는 제공않은 항목)

세부항목(item_code1): H2200 (transfer amount — 자금이체(금액), 단위: 억원)

분석 기간: 2000 (2000년) 부터 2023 (2023년) 까지 (연도 형식 YYYY)

요청 URL 예시 포맷(자동 생성):

https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/9999/{stat_code}/A/{start}/{end}/{item_code1}




---

## 작업 순서 (상세)

### 1) 데이터 수집
- 위 고정 파라미터로 ECOS API를 호출하여 JSON 응답을 받으세요.
- 응답에서 `row`(또는 통계 데이터 리스트)를 추출하여 DataFrame으로 변환하세요.
- 
### 2) 전처리 (CSV 저장까지)
1. **열 확인**  
 - 시간 컬럼(예: `TIME`)과 값 컬럼(예: `DATA_VALUE` 등)을 확인.
2. **시간 변환**  
 - `TIME` (YYYY) → `year` (int) 생성.  
 - `date` 컬럼 생성: `YYYY-01-01` 형식.  
 - `date`를 인덱스로 설정하고 오름차순 정렬.
3. **값 정리**  
 - 값 컬럼에서 콤마(,) 제거 후 숫자형으로 변환. 변환 불가 항목은 `NaN`.
4. **결측치 처리**  
 - `value`의 결측치는 **선형 보간(linear interpolation)** 으로 채움. (양끝 결측 포함 `limit_direction='both'`)  
 - 전처리 전/후 결측치 개수를 기록하여 보고서에 표기.
5. **보조 컬럼 생성**  
 - `value_interp` : 보간된 값(완성 시계열)  
 - `ma_3` : 3-year rolling mean (window=3, min_periods=1)  
 - `pct_yoy` : 연간 전년대비(%) = `(t - t-1) / t-1 * 100`
6. **기간별 평균 계산(수치 비교용)**  
 - `Recent 3-year mean` = 평균(`value_interp`, 2021–2023)  
 - `Previous 3-year mean` = 평균(`value_interp`, 2018–2020)  
 - 변화율(%) = `(recent - previous) / previous * 100`
7. **최신 관측치 비교**  
 - Latest year value (`value_interp` at max date)  
 - 1-year-ago value (latest year - 1)  
 - Latest YoY (%) 계산
8. **CSV 저장**  
 - 파일명: `ecos_631Y001_H2200_annual_cleaned.csv`  
 - 포함 컬럼: `TIME`, `year`, `date`, `value`, `value_interp`, `ma_3`, `pct_yoy` (이외 컬럼 포함 가능)

### 3) 시각화
1. **Time series plot** (`timeseries_internet_transfer_annual.png`)  
 - 그리기: `value_interp` (점 또는 선) + `ma_3` (선)를 겹쳐서 표시.  
 - Labels (영어): Title = `"Internet Banking — Transfer Amount (Annual)"`, x = `"Year"`, y = `"Amount (100 million KRW)"`.  
 - Legend 포함: e.g., `"Original (interpolated)"`, `"3-year moving average"`.  
 - 파일은 PNG 형식, 가로형 권장(최소 1200×600).
2. **Annual average bar chart**
 - 연도별 평균값(annual mean)을 막대그래프로 표시.  
 - Labels (영어): Title = `"Annual Average Amount by Year"`, x = `"Year"`, y = `"Average Amount (100 million KRW)"`.  

3. **월별 히트맵**  
 - 이 stat_code는 월별 데이터를 제공하지 않으므로 **월별 히트맵은 작성하지 않아도 됩니다.**

---

### 4) 제출 파일 목록
1. 전처리가 끝난 DataFrame을 ecos_631Y001_H2200_annual_cleaned.csv 로 저장

2. 연습문제1.ipynb

위 2개의 파일을 제출합니다.


### 결과 참고 이미지


<img src="https://imgur.com/vgHL5fe.png" width="80%">
<img src="https://imgur.com/WVVsePP.png" width="80%">
<img src="https://imgur.com/pHEhDUe.png" width="80%">








In [ ]:

import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (12,5)
sns.set_style('whitegrid')
